<h1 align="center">Pandas Homework - PyCitySchools</h1>

<text align="left">Created by: Luz del Carmen Vázquez Galindo </text>
<br>
<text align="left">10/10/2020</text>

In [131]:
import pandas as pd

#Create DataFrame by reading CSV file
schools_df = pd.read_csv("Resources/schools_complete.csv")
students_df = pd.read_csv("Resources/students_complete.csv")

In [132]:
#Merging both school and student tables and assigning it to a new DataFrame
district_df=schools_df.merge(students_df, how="left", on="school_name")


In [133]:
#Group By "school_name" variable 
school_sum = district_df.groupby("school_name",as_index=True)

In [134]:
#Functions Definition

#Returns the total number of students in a given DataFrame object
def get_total_Students(df):
    return len(pd.unique(df["Student ID"]))    

In [135]:
#Returns the total Budget using DataFrame.GroupBy "school_name_gb". 
#Receives a boolean to determine if all values in Series should be summarize
def get_total_Budget(isAll):
    total_budget=0
    
    if(isAll):
        total_bud = 0
        budget=school_sum["budget"].unique()
    
        for x in budget:
            total_bud = x + total_bud
        total_budget = total_bud[0]
    else:
        total_budget = school_sum["budget"].unique()

    return total_budget    

In [136]:
#Returns Average Score given a DataFrame and field name
def get_average_score(df, field_name):
    return df[field_name].mean()


In [137]:
#Tests whether student has the required grade to pass a given assignature using DataFrame object, returning DataFrame if true
def has_Passed_Subject(df,field_name, grade):
    return df.loc[df[field_name] >= grade].shape[0]


In [138]:
#Tests if and only if the student has passed two Subjects
def has_overall_passed(df, subject1, subject2, grade):
    return df.loc[(df[subject1] >= grade) & (df[subject2] >= grade) ].shape[0]

In [139]:
#Returns DataFrame containing students' scores in Math, Reading and both given a bin to group by
def get_score_by(score_by_df, new_field, cut_field, bins, group_labels):
          
    #Adding variables to display in result DataFrame
    score_avg_math = []
    score_avg_read = []
    score_pass_math = []
    score_pass_read = []
    score_pass_overall = []
        
    #Validating whether new_field is '' or not. If '', there's no need to use bins
    if new_field =='':
            score_by_gbdf = score_by_df.groupby(cut_field)
    else:
        score_by_df[new_field]=pd.cut(score_by_df[cut_field], bins, labels=group_labels, include_lowest=True) 
        score_by_gbdf = score_by_df.groupby(new_field)
        
    
    school_type_df.groupby("School Type")
    

    #Iterating on each tuple of score bin and data
    for score in score_by_gbdf:
        score_avg_math.append("{0:.2f}".format(get_average_score(score[1],'Average Math Score')))
        score_avg_read.append("{0:.2f}".format(get_average_score(score[1],'Average Reading Score')))
        score_pass_math.append("{0:.2f}".format(get_average_score(score[1],'%Passing Math')))
        score_pass_read.append("{0:.2f}".format(get_average_score(score[1],'%Passing Reading')))
        score_pass_overall.append("{0:.2f}".format(get_average_score(score[1],'%Overall Passing')))

    #Creating DataFram with results to print
    score_by_index = score_by_gbdf.groups.keys()
    
    score_by = pd.DataFrame({
    'Average Math Score':score_avg_math,
    'Average Reading Score':score_avg_read,
    '% Passing Math':score_pass_math,
    '% Passing Reading': score_pass_read,
    '% Overall Passing': score_pass_overall
    }, index=score_by_index)
                                                                    
    return score_by

<h2>District Summary</h2>

Create a high level snapshot (in table form) of the district's key metrics, including:

<UL>
<LI>Total Schools
<LI>Total Students
<LI>Total Budget
<LI>Average Math Score
<LI>Average Reading Score
<LI>% Passing Math (The percentage of students that passed math.)
<LI>% Passing Reading (The percentage of students that passed reading.)
<LI>% Overall Passing (The percentage of students that passed math and reading.)
</UL>

In [140]:
#Creating variables for each District Summary value

#Total Schools
total_schools = len(pd.unique(district_df["school_name"]))
total_schools

15

In [141]:
#Total Students
total_students = get_total_Students(district_df)
total_students

39170

In [142]:
#Total Budget
total_budget = get_total_Budget(True)
total_budget

24649428

In [143]:
#Average Math Score
avg_math_score = get_average_score(district_df, "math_score")
avg_math_score

78.98537145774827

In [144]:
#Average Reading Score
avg_read_score = get_average_score(district_df, "reading_score")
avg_read_score

81.87784018381414

In [145]:
#% Passing Math (The percentage of students that passed math.)
pass_math=(has_Passed_Subject(district_df,"math_score", 70)/total_students)*100
pass_math

74.9808526933878

In [146]:
#% Passing Reading (The percentage of students that passed reading.)
pass_read=(has_Passed_Subject(district_df,"reading_score", 70)/total_students)*100
pass_read

85.80546336482001

In [147]:
#% Overall Passing (The percentage of students that passed math and reading.)
pass_overall=(has_overall_passed(district_df, "math_score", "reading_score",70)/total_students)*100
pass_overall

65.17232575950983

In [148]:
#Creating new Data Frame with results
data ={
    "Total Schools":[total_schools], 
    "Total Students":[total_students], 
    "Total Budget":[total_budget], 
    "Average Math Score": [avg_math_score], 
    "Average Reading Score": [avg_read_score], 
    "% Passing Math": [pass_math],
    "% Passing Reading": [pass_read], 
    "% Overall Passing": [pass_overall]
}

district_summary_df = pd.DataFrame(data=data)
format_dict = {'Total Students':'{0:,.0}', 'Total Budget': '${0:,.0f}'}

district_summary_df.style.format(format_dict)

district_summary_df.head()

,Total Schools,Total Students,Total Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing
0,15,39170,24649428,78.985371,81.87784,74.980853,85.805463,65.172326


<h2>School Summary</h2>

<ul>
    <li>Create an overview table that summarizes key metrics about each school, including:</li>
    <ul>
<li>School Name</li>
<li>School Type</li>
<li>Total Students</li>
<li>Total School Budget</li>
<li>Per Student Budget</li>
<li>Average Math Score</li>
<li>Average Reading Score</li>
<li>% Passing Math</li>
<li>% Passing Reading</li>
<li>% Overall Passing (The percentage of students that passed math and reading.)</li>
    </ul>
    <li>Create a dataframe to hold the above results</li>
 </ul> 

In [149]:
#Defining required values as Arrays

#Using school_sum DataFrameGroupBy object, all Data Frame groups (schools) are gotten to get each school's requested values
school_names=school_sum.groups.keys()

#The rest of arrays are only initialized as empty
school_type = []
school_total_students = []
school_total_budget = []
school_per_student_budget = []
school_avg_math_score = []
school_avg_read_score = []
school_pass_math = []
school_pass_read = []
school_pass_overall = []

In [150]:
#DataFrameGroupBy get_group method is used to get each of the 15 Data Frames. School_names list is used to get them
for school_name in school_names:
    school_df=school_sum.get_group(school_name)
    
    school_total_students_num = school_df["size"].unique()[0]
    school_total_budget_num = school_df["budget"].unique()[0]
    
    school_type.append(school_df['type'].unique()[0])    
    school_total_students.append(school_total_students_num)
    school_total_budget.append(school_total_budget_num)
    school_per_student_budget.append(school_total_budget_num/school_total_students_num)
    school_avg_math_score.append(get_average_score(school_df,"math_score"))
    school_avg_read_score.append(get_average_score(school_df,"reading_score"))
    school_pass_math.append((has_Passed_Subject(school_df,"math_score", 70)/school_total_students_num)*100)
    school_pass_read.append((has_Passed_Subject(school_df,"reading_score", 70)/school_total_students_num)*100)
    school_pass_overall.append((has_overall_passed(school_df, "math_score", "reading_score",70)/school_total_students_num)*100)

In [151]:
school_summary = pd.DataFrame({
    "School Type":school_type,
    "Total Students":school_total_students,
    "Total School Budget":school_total_budget,
    "Per Student Budget":school_per_student_budget,
    "Average Math Score":school_avg_math_score,
    "Average Reading Score":school_avg_read_score,
    "%Passing Math":school_pass_math,
    "%Passing Reading":school_pass_read,
    "%Overall Passing":school_pass_overall   
}, index=school_names)

school_summary

,School Type,Total Students,Total School Budget,Per Student Budget,Average Math Score,Average Reading Score,%Passing Math,%Passing Reading,%Overall Passing
Bailey High School,District,4976,3124928,628.0,77.048432,81.033963,66.680064,81.933280,54.642283
Cabrera High School,Charter,1858,1081356,582.0,83.061895,83.975780,94.133477,97.039828,91.334769
Figueroa High School,District,2949,1884411,639.0,76.711767,81.158020,65.988471,80.739234,53.204476
Ford High School,District,2739,1763916,644.0,77.102592,80.746258,68.309602,79.299014,54.289887
Griffin High School,Charter,1468,917500,625.0,83.351499,83.816757,93.392371,97.138965,90.599455
Hernandez High School,District,4635,3022020,652.0,77.289752,80.934412,66.752967,80.862999,53.527508
Holden High School,Charter,427,248087,581.0,83.803279,83.814988,92.505855,96.252927,89.227166
Huang High School,District,2917,1910635,655.0,76.629414,81.182722,65.683922,81.316421,53.513884
Johnson High School,District,4761,3094650,650.0,77.072464,80.966394,66.057551,81.222432,53.539172
Pena High School,Charter,962,585858,609.0,83.839917,84.044699,94.594595,95.945946,90.540541


## Top Performing Schools (By % Overall Passing)

* Sort and display the top five performing schools by % overall passing.

In [152]:
school_summary.sort_values(by='%Overall Passing', ascending=False).head(5)

,School Type,Total Students,Total School Budget,Per Student Budget,Average Math Score,Average Reading Score,%Passing Math,%Passing Reading,%Overall Passing
Cabrera High School,Charter,1858,1081356,582.0,83.061895,83.975780,94.133477,97.039828,91.334769
Thomas High School,Charter,1635,1043130,638.0,83.418349,83.848930,93.272171,97.308869,90.948012
Griffin High School,Charter,1468,917500,625.0,83.351499,83.816757,93.392371,97.138965,90.599455
Wilson High School,Charter,2283,1319574,578.0,83.274201,83.989488,93.867718,96.539641,90.582567
Pena High School,Charter,962,585858,609.0,83.839917,84.044699,94.594595,95.945946,90.540541


## Bottom Performing Schools (By % Overall Passing)

* Sort and display the five worst-performing schools by % overall passing.

In [153]:
school_summary.sort_values(by='%Overall Passing', ascending=True).head(5)

,School Type,Total Students,Total School Budget,Per Student Budget,Average Math Score,Average Reading Score,%Passing Math,%Passing Reading,%Overall Passing
Rodriguez High School,District,3999,2547363,637.0,76.842711,80.744686,66.366592,80.220055,52.988247
Figueroa High School,District,2949,1884411,639.0,76.711767,81.158020,65.988471,80.739234,53.204476
Huang High School,District,2917,1910635,655.0,76.629414,81.182722,65.683922,81.316421,53.513884
Hernandez High School,District,4635,3022020,652.0,77.289752,80.934412,66.752967,80.862999,53.527508
Johnson High School,District,4761,3094650,650.0,77.072464,80.966394,66.057551,81.222432,53.539172


## Math Scores by Grade

* Create a table that lists the average Math Score for students of each grade level (9th, 10th, 11th, 12th) at each school.

  * Create a pandas series for each grade. Hint: use a conditional statement.
  
  * Group each series by school
  
  * Combine the series into a dataframe
  
  * Optional: give the displayed data cleaner formatting

In [154]:
#Creating the Series for each grade
grade_9th_s = district_df.loc[district_df['grade']=='9th']
grade_10th_s = district_df.loc[district_df['grade']=='10th']
grade_11th_s = district_df.loc[district_df['grade']=='11th']
grade_12th_s = district_df.loc[district_df['grade']=='12th']


In [155]:
#Grouping by School and getting average Math grade on each school
#Setting school_name as index
grade_9th_m_avg = grade_9th_s.groupby("school_name", as_index=True)["math_score"].mean()
grade_10th_m_avg = grade_10th_s.groupby("school_name", as_index=True)["math_score"].mean()
grade_11th_m_avg = grade_11th_s.groupby("school_name", as_index=True)["math_score"].mean()
grade_12th_m_avg = grade_12th_s.groupby("school_name", as_index=True)["math_score"].mean()

In [156]:
#Getting keys in any of the series (taken 9th grade as a sample) to define index
grade_math_avg_index = grade_9th_m_avg.keys()

grade_math_avg_df = pd.DataFrame({
    '9th':grade_9th_m_avg,
    '10th':grade_10th_m_avg,
    '11th':grade_11th_m_avg,
    '12th':grade_12th_m_avg
}, index=grade_math_avg_index)

grade_math_avg_df

,9th,10th,11th,12th
school_name,,,,
Bailey High School,77.083676,76.996772,77.515588,76.492218
Cabrera High School,83.094697,83.154506,82.765560,83.277487
Figueroa High School,76.403037,76.539974,76.884344,77.151369
Ford High School,77.361345,77.672316,76.918058,76.179963
Griffin High School,82.044010,84.229064,83.842105,83.356164
Hernandez High School,77.438495,77.337408,77.136029,77.186567
Holden High School,83.787402,83.429825,85.000000,82.855422
Huang High School,77.027251,75.908735,76.446602,77.225641
Johnson High School,77.187857,76.691117,77.491653,76.863248


## Reading Score by Grade 

* Perform the same operations as above for reading scores

In [157]:
#Using the series created in previous exercise (grade_9th_s, grade_10th_s, grade_11th_s, and grade_12th_s), so there's no
#need to create new ones

#Grouping by School and getting average Reading grade on each school
#Setting school_name as index
grade_9th_r_avg = grade_9th_s.groupby("school_name", as_index=True)["reading_score"].mean()
grade_10th_r_avg = grade_10th_s.groupby("school_name", as_index=True)["reading_score"].mean()
grade_11th_r_avg = grade_11th_s.groupby("school_name", as_index=True)["reading_score"].mean()
grade_12th_r_avg = grade_12th_s.groupby("school_name", as_index=True)["reading_score"].mean()

In [158]:
#Getting keys in any of the series (taken 9th grade as a sample) to define index
grade_read_avg_index = grade_9th_r_avg.keys()

grade_read_avg_df = pd.DataFrame({
    '9th':grade_9th_r_avg,
    '10th':grade_10th_r_avg,
    '11th':grade_11th_r_avg,
    '12th':grade_12th_r_avg
}, index=grade_read_avg_index)

grade_read_avg_df

,9th,10th,11th,12th
school_name,,,,
Bailey High School,81.303155,80.907183,80.945643,80.912451
Cabrera High School,83.676136,84.253219,83.788382,84.287958
Figueroa High School,81.198598,81.408912,80.640339,81.384863
Ford High School,80.632653,81.262712,80.403642,80.662338
Griffin High School,83.369193,83.706897,84.288089,84.013699
Hernandez High School,80.866860,80.660147,81.396140,80.857143
Holden High School,83.677165,83.324561,83.815534,84.698795
Huang High School,81.290284,81.512386,81.417476,80.305983
Johnson High School,81.260714,80.773431,80.616027,81.227564


## Scores by School Spending

* Create a table that breaks down school performances based on average Spending Ranges (Per Student). Use 4 reasonable bins to group school spending. Include in the table each of the following:
  * Average Math Score
  * Average Reading Score
  * % Passing Math
  * % Passing Reading
  * Overall Passing Rate (Average of the above two)

In [159]:
#Copying district_df to add columns in other data frame
school_spending_df = school_summary.copy()

#Adding a columns to district_summary using bins on size
school_spending_bins = [0,584,629,644,675]
school_spending_labels = ['<$585','$585-629', '$630-644', '$645-675']

#Getting DF to print results in a table
score_by_school_spending=get_score_by(school_spending_df, 'Spending Score', 'Per Student Budget', school_spending_bins, school_spending_labels)

#Printing results
score_by_school_spending

,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing
<$585,83.46,83.93,93.46,96.61,90.37
$585-629,81.90,83.16,87.13,92.72,81.42
$630-644,78.52,81.62,73.48,84.39,62.86
$645-675,77.00,81.03,66.16,81.13,53.53


## Scores by School Size

* Perform the same operations as above, based on school size.

In [160]:
#Copying district_df to add columns in other data frame
school_size_df = school_summary.copy()

#Adding a columns to district_summary using bins on size
school_size_bins = [0,999,2000,5000]
school_size_labels = ['Small (<1000)','Medium (1000-2000)', 'Large (2000-5000)']

#Getting DF to print results in a table
score_by_school_size=get_score_by(school_size_df, 'School Size', 'Total Students', school_size_bins, school_size_labels)

#Printing results
score_by_school_size

,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing
Small (<1000),83.82,83.93,93.55,96.10,89.88
Medium (1000-2000),83.37,83.86,93.60,96.79,90.62
Large (2000-5000),77.75,81.34,69.96,82.77,58.29


## Scores by School Type

* Perform the same operations as above, based on school type

In [161]:
#Copying district_df to add columns in other data frame
school_type_df = school_summary.copy()

#Getting DF to print results in a table. In function, 'new_field', 'bins', 'group_labels' are empty, 
#as there's no need to use bins to group by
score_by_school_type = get_score_by(school_type_df, '', 'School Type', [], [])

#Printing results
score_by_school_type

,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing
Charter,83.47,83.90,93.62,96.59,90.43
District,76.96,80.97,66.55,80.80,53.67
